# Exercise (part 1) - implementation of FBA problem
## 3.
## a. Evaluate if single gene deletions enhance the production of the compound. <br> Rank the mutants obtained according to the compound production capacity and growth performance. 
### 1º Simulação FBA para saber a taxa de crescimento antes das deleções de genes, ou seja, taxa de crescimento do wild-type <br>
**Model objective - uma vez que temos que considerar que as células evoluíram para cresciento máximo, queremos maximizar a biomassa e, por isso, temos que mudar de 1 para -1; e tbm queremos maximizar a produção do nosso composto (lactato).**

In [9]:
import cobra

import pandas
from time import time

from cobra.util.solver import linear_reaction_coefficients

import cobra.test
from cobra.flux_analysis import (single_gene_deletion, single_reaction_deletion, double_gene_deletion, double_reaction_deletion)

cobramo = cobra.io.read_sbml_model('iML1515.xml')

cobramo.objective = {cobramo.reactions.BIOMASS_Ec_iML1515_core_75p37M: -1.0}
cobramo.objective

cobramo.reactions.get_by_id("EX_o2_e").upper_bound = 1000
cobramo.reactions.get_by_id("EX_o2_e").lower_bound = 0
cobramo.reactions.get_by_id("EX_glc__D_e").upper_bound = 1000
cobramo.reactions.get_by_id("EX_glc__D_e").lower_bound = -15

solution = cobramo.optimize()
print('FBA atraves do cobra:', solution)
print('taxa de crescimento antes da maximização da produção do composto:', solution.objective_value)
print('fluxo da reação da biomassa:', solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

cobramo.objective = {cobramo.reactions.EX_lac__L_e: -1.0, cobramo.reactions.EX_lac__D_e: -1.0}
cobramo.objective

solutionopt = cobramo.optimize()
print('FBA atraves do cobra:', solutionopt)
print('taxa de crescimento depois da maximização da produção do composto:', solutionopt.objective_value)
print('fluxo da reação da biomassa:', solutionopt.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

FBA atraves do cobra: <Solution 0.000 at 0x7f32f6244510>
taxa de crescimento antes da maximização da produção do composto: 0.0
fluxo da reação da biomassa: 0.0
FBA atraves do cobra: <Solution 0.000 at 0x7f32f3e9b110>
taxa de crescimento depois da maximização da produção do composto: 0.0
fluxo da reação da biomassa: 0.0


### 2º (wild type) Reações que (SÓ) produzem  lactato e fluxos delas:

In [10]:
print('R_EX_lac__L_e', solutionopt.fluxes['EX_lac__L_e'])
print('R_EX_lac__D_e', solutionopt.fluxes['EX_lac__D_e'])

R_EX_lac__L_e 0.0
R_EX_lac__D_e 0.0


### 3º All single gene deletions

In [13]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 2000)

deletion_results = single_gene_deletion(cobramo)
print(deletion_results)

          ids  growth      status
0     {b3713}     0.0     optimal
1     {b1124}     0.0     optimal
2     {b0614}     0.0     optimal
3     {b0763}     0.0     optimal
4     {b1263}     0.0     optimal
5     {b4209}     0.0     optimal
6     {b4358}     0.0     optimal
7     {b2392}     0.0     optimal
8     {b1276}     0.0     optimal
9     {b2995}     0.0     optimal
10    {b2366}     0.0     optimal
11    {b4390}     0.0     optimal
12    {b2291}     0.0     optimal
13    {b1260}     0.0     optimal
14    {b0349}     0.0     optimal
15    {b0131}     0.0     optimal
16    {b2484}     0.0     optimal
17    {b1398}     0.0     optimal
18    {b4055}     0.0     optimal
19    {b4130}     0.0     optimal
20    {b2779}     NaN  infeasible
21    {b3752}     0.0     optimal
22    {b2430}     0.0     optimal
23    {b3005}     0.0     optimal
24    {b2964}     0.0     optimal
25    {b4099}     0.0     optimal
26    {b3091}     0.0     optimal
27    {b0369}     0.0     optimal
28    {b2683} 

### 4º genes essenciais (não podem ser eliminados):

In [4]:
from reframed.io.sbml import load_cbmodel
model = load_cbmodel('iML1515.xml', flavor = 'fbc2')

from mewpy.simulation import get_simulator
simul = get_simulator(model)

envcond = {'R_EX_o2_e':0, 'R_EX_glc__D_e':-15}

simul = get_simulator(model, envcond = envcond)

simul.objective = {'R_BIOMASS_Ec_iML1515_core_75p37M': -1.0, 'R_EX_lac__L_e': -1.0, 'R_EX_lac__D_e': -1.0}

result = simul.simulate(method='FBA')
print("FBA atraves do reframed:")
print(result)
print(result.fluxes['R_BIOMASS_Ec_iML1515_core_75p37M'])
print(result.fluxes['R_EX_lac__L_e'])
print(result.fluxes['R_EX_lac__D_e'])

essent_genes = simul.essential_genes
print(essent_genes)

/usr/local/lib/python3.7/site-packages/reframed/io/sbml.py:510: UserWarning: Gene association for reaction R_PFL is not DNF
  warn(f"Gene association for reaction {reaction_id} is not DNF")
/usr/local/lib/python3.7/site-packages/reframed/io/sbml.py:510: UserWarning: Gene association for reaction R_RNDR1 is not DNF
  warn(f"Gene association for reaction {reaction_id} is not DNF")
/usr/local/lib/python3.7/site-packages/reframed/io/sbml.py:510: UserWarning: Gene association for reaction R_RNDR3 is not DNF
  warn(f"Gene association for reaction {reaction_id} is not DNF")
/usr/local/lib/python3.7/site-packages/reframed/io/sbml.py:510: UserWarning: Gene association for reaction R_RNDR4 is not DNF
  warn(f"Gene association for reaction {reaction_id} is not DNF")
/usr/local/lib/python3.7/site-packages/reframed/io/sbml.py:510: UserWarning: Gene association for reaction R_CITL is not DNF
  warn(f"Gene association for reaction {reaction_id} is not DNF")
/usr/local/lib/python3.7/site-packages/refr

FBA atraves do reframed:
objective: 0.0
Status: OPTIMAL
0.0
0.0
0.0
['G_b2779', 'G_b2926', 'G_b1779']


### 5º Iterar sobre cada gene que está no deletion_results, sempre que aparecer um gene essencial continuar o loop e depois apagar gene a gene do modelo. Após tirar um gene, fazer a simulação FBA e calcular os fluxos das reações que produzem lactato. No fim, voltar a colocar o modelo com os genes todos.

In [5]:
import pandas
from time import time

import cobra.test

import cobra

from mewpy.simulation import get_simulator

for id in deletion_results.ids:
    for gene in id:
        genelist = [str(gene)]
        if gene not in essent_genes:
            print(genelist)
            cobra.manipulation.remove_genes(cobramo, genelist)
            solutionko = cobramo.optimize()
            solutionko.objective_value
            print('produção da lactose', solutionko.objective_value)
            
            cobramo = cobra.io.read_sbml_model('iML1515.xml')
            
            cobramo.objective = {cobramo.reactions.BIOMASS_Ec_iML1515_core_75p37M: -1.0}
            cobramo.objective

            cobramo.reactions.get_by_id("EX_o2_e").upper_bound = 1000
            cobramo.reactions.get_by_id("EX_o2_e").lower_bound = 0
            cobramo.reactions.get_by_id("EX_glc__D_e").upper_bound = 1000
            cobramo.reactions.get_by_id("EX_glc__D_e").lower_bound = -15

            cobra.manipulation.remove_genes(cobramo, genelist)
            solutionkobiomass = cobramo.optimize()
            print('growth rate', solutionkobiomass.objective_value)
            
            cobramo = cobra.io.read_sbml_model('iML1515.xml')

            cobramo.objective = {cobramo.reactions.BIOMASS_Ec_iML1515_core_75p37M: -1.0}
            cobramo.objective

            cobramo.reactions.get_by_id("EX_o2_e").upper_bound = 1000
            cobramo.reactions.get_by_id("EX_o2_e").lower_bound = 0
            cobramo.reactions.get_by_id("EX_glc__D_e").upper_bound = 1000
            cobramo.reactions.get_by_id("EX_glc__D_e").lower_bound = -15

            solution = cobramo.optimize()

            cobramo.objective = {cobramo.reactions.EX_lac__L_e: -1.0, cobramo.reactions.EX_lac__D_e: -1.0}
            cobramo.objective            


['b2162']
produção da lactose 0.0
growth rate 0.0
['b2536']
produção da lactose 0.0
growth rate 0.0
['b3365']
produção da lactose 0.0
growth rate 0.0
['b3708']
produção da lactose 0.0
growth rate 0.0
['b4383']
produção da lactose 0.0
growth rate 0.0
['b0221']
produção da lactose 0.0
growth rate 0.0
['b2341']
produção da lactose 0.0
growth rate 0.0
['b1732']
produção da lactose 0.0
growth rate 0.0
['b1640']
produção da lactose 0.0
growth rate 0.0
['b2094']
produção da lactose 0.0
growth rate 0.0
['b4019']
produção da lactose 0.0
growth rate 0.0
['b0750']
produção da lactose 0.0
growth rate 0.0
['b4238']
produção da lactose 0.0
growth rate 0.0
['b3735']
produção da lactose 0.0
growth rate 0.0
['b3266']
produção da lactose 0.0
growth rate 0.0
['b2918']
produção da lactose 0.0
growth rate 0.0
['b1012']
produção da lactose 0.0
growth rate 0.0
['b3032']
produção da lactose 0.0
growth rate 0.0
['b2323']
produção da lactose 0.0
growth rate 0.0
['b0805']
produção da lactose 0.0
growth rate 0.0


['b1677']
produção da lactose 0.0
growth rate 0.0
['b3248']
produção da lactose 0.0
growth rate 0.0
['b3572']
produção da lactose 0.0
growth rate 0.0
['b0355']
produção da lactose 0.0
growth rate 0.0
['b3040']
produção da lactose 0.0
growth rate 0.0
['b4477']
produção da lactose 0.0
growth rate 0.0
['b3425']
produção da lactose 0.0
growth rate 0.0
['b0261']
produção da lactose 0.0
growth rate 0.0
['b0150']
produção da lactose 0.0
growth rate 0.0
['b1517']
produção da lactose 0.0
growth rate 0.0
['b3640']
produção da lactose 0.0
growth rate 0.0
['b0040']
produção da lactose 0.0
growth rate 0.0
['b0577']
produção da lactose 0.0
growth rate 0.0
['b2038']
produção da lactose 0.0
growth rate 0.0
['b0353']
produção da lactose 0.0
growth rate 0.0
['b4093']
produção da lactose 0.0
growth rate 0.0
['b2406']
produção da lactose 0.0
growth rate 0.0
['b1098']
produção da lactose 0.0
growth rate 0.0
['b2476']
produção da lactose 0.0
growth rate 0.0
['b2235']
produção da lactose 0.0
growth rate 0.0


['b1870']
produção da lactose 0.0
growth rate 0.0
['b2663']
produção da lactose 0.0
growth rate 0.0
['b3671']
produção da lactose 0.0
growth rate 0.0
['b1513']
produção da lactose 0.0
growth rate 0.0
['b3057']
produção da lactose 0.0
growth rate 0.0
['b0766']
produção da lactose 0.0
growth rate 0.0
['b1852']
produção da lactose 0.0
growth rate 0.0
['b2558']
produção da lactose 0.0
growth rate 0.0
['b4384']
produção da lactose 0.0
growth rate 0.0
['b2913']
produção da lactose 0.0
growth rate 0.0
['b0182']
produção da lactose 0.0
growth rate 0.0
['b2039']
produção da lactose 0.0
growth rate 0.0
['b1636']
produção da lactose 0.0
growth rate 0.0
['b1317']
produção da lactose 0.0
growth rate 0.0
['b1010']
produção da lactose 0.0
growth rate 0.0
['b4291']
produção da lactose 0.0
growth rate 0.0
['b1395']
produção da lactose 0.0
growth rate 0.0
['b2752']
produção da lactose 0.0
growth rate 0.0
['b3114']
produção da lactose 0.0
growth rate 0.0
['b2817']
produção da lactose 0.0
growth rate 0.0


['b2871']
produção da lactose 0.0
growth rate 0.0
['b0036']
produção da lactose 0.0
growth rate 0.0
['b1686']
produção da lactose 0.0
growth rate 0.0
['b3475']
produção da lactose 0.0
growth rate 0.0
['b2813']
produção da lactose 0.0
growth rate 0.0
['b3617']
produção da lactose 0.0
growth rate 0.0
['b3469']
produção da lactose 0.0
growth rate 0.0
['b0207']
produção da lactose 0.0
growth rate 0.0
['b0054']
produção da lactose 0.0
growth rate 0.0
['b0120']
produção da lactose 0.0
growth rate 0.0
['b1062']
produção da lactose 0.0
growth rate 0.0
['b4394']
produção da lactose 0.0
growth rate 0.0
['b2174']
produção da lactose 0.0
growth rate 0.0
['b1662']
produção da lactose 0.0
growth rate 0.0
['b1678']
produção da lactose 0.0
growth rate 0.0
['b1008']
produção da lactose 0.0
growth rate 0.0
['b1991']
produção da lactose 0.0
growth rate 0.0
['b1476']
produção da lactose 0.0
growth rate 0.0
['b1197']
produção da lactose 0.0
growth rate 0.0
['b1740']
produção da lactose 0.0
growth rate 0.0


['b3957']
produção da lactose 0.0
growth rate 0.0
['b4244']
produção da lactose 0.0
growth rate 0.0
['b4131']
produção da lactose 0.0
growth rate 0.0
['b3397']
produção da lactose 0.0
growth rate 0.0
['b2912']
produção da lactose 0.0
growth rate 0.0
['b4006']
produção da lactose 0.0
growth rate 0.0
['b2153']
produção da lactose 0.0
growth rate 0.0
['b4177']
produção da lactose 0.0
growth rate 0.0
['b3073']
produção da lactose 0.0
growth rate 0.0
['b2373']
produção da lactose 0.0
growth rate 0.0
['b4115']
produção da lactose 0.0
growth rate 0.0
['b3374']
produção da lactose 0.0
growth rate 0.0
['b4161']
produção da lactose 0.0
growth rate 0.0
['b0469']
produção da lactose 0.0
growth rate 0.0
['b1378']
produção da lactose 0.0
growth rate 0.0
['b0025']
produção da lactose 0.0
growth rate 0.0
['b0655']
produção da lactose 0.0
growth rate 0.0
['b2934']
produção da lactose 0.0
growth rate 0.0
['b3825']
produção da lactose 0.0
growth rate 0.0
['b0632']
produção da lactose 0.0
growth rate 0.0


...

/usr/local/lib/python3.7/site-packages/cobra/util/solver.py:508: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


produção da lactose 0.03298076923076924
growth rate 0.032980769230769244
['b2997']
produção da lactose 0.0
growth rate 0.0
['b1621']
produção da lactose 0.0
growth rate 0.0
['b4468']
produção da lactose 0.0
growth rate 0.0
['b1469']
produção da lactose 0.0
growth rate 0.0
['b2258']
produção da lactose 0.0
growth rate 0.0
['b3670']
produção da lactose 0.0
growth rate 0.0
['b1511']
produção da lactose 0.0
growth rate 0.0
['b0383']
produção da lactose 0.0
growth rate 0.0
['b2129']
produção da lactose 0.0
growth rate 0.0
['b3581']
produção da lactose 0.0
growth rate 0.0
['b1232']
produção da lactose 0.0
growth rate 0.0
['b0933']
produção da lactose 0.0
growth rate 0.0
['b0586']
produção da lactose 0.0
growth rate 0.0
['b2374']
produção da lactose 0.0
growth rate 0.0
['b3213']
produção da lactose 0.0
growth rate 0.0
['b1123']
produção da lactose 0.0
growth rate 0.0
['b3265']
produção da lactose 0.0
growth rate 0.0
['b4013']
produção da lactose 0.0
growth rate 0.0
['b3431']
produção da lacto

['b0781']
produção da lactose 0.0
growth rate 0.0
['b0856']
produção da lactose 0.0
growth rate 0.0
['b2065']
produção da lactose 0.0
growth rate 0.0
['b0615']
produção da lactose 0.0
growth rate 0.0
['b3966']
produção da lactose 0.0
growth rate 0.0
['b0755']
produção da lactose 0.0
growth rate 0.0
['b0341']
produção da lactose 0.0
growth rate 0.0
['b1634']
produção da lactose 0.0
growth rate 0.0
['b2797']
produção da lactose 0.0
growth rate 0.0
['b0764']
produção da lactose 0.0
growth rate 0.0
['b2789']
produção da lactose 0.0
growth rate 0.0
['b2428']
produção da lactose 0.0
growth rate 0.0
['b3583']
produção da lactose 0.0
growth rate 0.0
['b1385']
produção da lactose 0.0
growth rate 0.0
['b3565']
produção da lactose 0.0
growth rate 0.0
['b1440']
produção da lactose 0.0
growth rate 0.0
['b1198']
produção da lactose 0.0
growth rate 0.0
['b3737']
produção da lactose 0.0
growth rate 0.0
['b4086']
produção da lactose 0.0
growth rate 0.0
['b4297']
produção da lactose 0.0
growth rate 0.0


['b0937']
produção da lactose 0.0
growth rate 0.0
['b1207']
produção da lactose 0.0
growth rate 0.0
['b4289']
produção da lactose 0.0
growth rate 0.0
['b3664']
produção da lactose 0.0
growth rate 0.0
['b3990']
produção da lactose 0.0
growth rate 0.0
['b0429']
produção da lactose 0.0
growth rate 0.0
['b0493']
produção da lactose 0.0
growth rate 0.0
['b0973']
produção da lactose 0.0
growth rate 0.0
['b3786']
produção da lactose 0.0
growth rate 0.0
['b2587']
produção da lactose 0.0
growth rate 0.0
['b3903']
produção da lactose 0.0
growth rate 0.0
['b2066']
produção da lactose 0.0
growth rate 0.0
['b3451']
produção da lactose 0.0
growth rate 0.0
['b3904']
produção da lactose 0.0
growth rate 0.0
['b3728']
produção da lactose 0.0
growth rate 0.0
['b2747']
produção da lactose 0.0
growth rate 0.0
['b0115']
produção da lactose 0.0
growth rate 0.0
['b1619']
produção da lactose 0.0
growth rate 0.0
['b2979']
produção da lactose 0.0
growth rate 0.0
['b2781']
produção da lactose 0.0
growth rate 0.0


['b4005']
produção da lactose 0.0
growth rate 0.0
['b4077']
produção da lactose 0.0
growth rate 0.0
['b1396']
produção da lactose 0.0
growth rate 0.0
['b2965']
produção da lactose 0.0
growth rate 0.0
['b1684']
produção da lactose 0.0
growth rate 0.0
['b0767']
produção da lactose 0.0
growth rate 0.0
['b0543']
produção da lactose 0.0
growth rate 0.0
['b3614']
produção da lactose 0.0
growth rate 0.0
['b2563']
produção da lactose 0.0
growth rate 0.0
['b2155']
produção da lactose 0.0
growth rate 0.0
['b0091']
produção da lactose 0.0
growth rate 0.0
['b3748']
produção da lactose 0.0
growth rate 0.0
['b3996']
produção da lactose 0.0
growth rate 0.0
['b3457']
produção da lactose 0.0
growth rate 0.0
['b3812']
produção da lactose 0.0
growth rate 0.0
['b1779']
produção da lactose 0.03298076923076925
growth rate 0.03298076923076924
['b2843']
produção da lactose 0.0
growth rate 0.0
['b4141']
produção da lactose 0.0
growth rate 0.0
['b2723']
produção da lactose 0.0
growth rate 0.0
['b0315']
produção